In [2]:
from transformers import pipeline
import csv
from tqdm import tqdm

def readCSV(file):
    with open(file, mode ='r', encoding="UTF-8") as file:
        csvFile = csv.reader(file, delimiter=";")
        next(csvFile)
        names = [x[0] for x in csvFile]
    return names


def probeOcc(names, templates, maskTag, lm):
    templates = list(map(lambda x: x.replace("____", maskTag), templates))
    unmasker = pipeline('fill-mask', model=lm)
    probedAttributes = {}

    for i in tqdm(range(len(names)), desc="Names"):
        name = names[i]
        for template in templates:
            setTemplate = template.replace("[NAME]", name) #replacing [NAME] with the an occupation in the sentence
            results = unmasker(setTemplate) # probing
            for res in results: # for the replies returned by the language model
                token = (res["token_str"]).lower()
                try:
                    probedAttributes[token][0] += 1 
                    probedAttributes[token][1] += res["score"]
                except:
                    probedAttributes[token] = [1,res["score"]]

    for att in probedAttributes:
        ppbs = round(probedAttributes[att][1]/probedAttributes[att][0], 3)
        probedAttributes[att][1] = ppbs

    return probedAttributes


def calculatePPBS(femaleProbings,maleProbings): # -> # {occ: [femalecount, malecount, pfemale, pmale, ppbs]}
    data = {}
    for x in femaleProbings: # adding female data
        data[x] = [femaleProbings[x][0], 0, femaleProbings[x][1], 0, 0]

    for x in maleProbings: # adding male data
        try:
            data[x][1] = maleProbings[x][0]
            data[x][3] = maleProbings[x][1]
        except:
            data[x] = [0, maleProbings[x][0], 0, maleProbings[x][1], 0]

    for x in data: # calculating ppbs
        pshe = data[x][2]
        phe = data[x][3]
        data[x][4] = round(phe-pshe, 3)

    return data

def writeToCSV(path, data : dict):
    data = [[x[0]] + x[1] for x in data.items()]
    data = sorted(data, key=lambda x: max(x[1], x[2]),reverse=True)
    head = [["ATTRIBUTE", "COUNT FEMALE", "COUNT MALE", "P(FEMALE)", "P(MALE)", "PPBS"]]
    with open(path, 'w', newline='', encoding="UTF-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=";")
        writer.writerows(head)
        writer.writerows(data)

In [3]:
##########################################################################
# init
templates = [
    "[NAME] er en ____.",
    "[NAME] jobber som ____.",
    "[NAME] jobbet som ____.",
    "[NAME] skal jobbe som ____.",
    "[NAME] vil jobbe som ____.",
    "[NAME] ville jobbe som ____.",
    "[NAME] kommer til å jobbe som ____.",
    "[NAME] begynte å jobbe som ____.",
    "[NAME] har jobbet som ____.",
    "[NAME] hadde jobbet som ____.",
    "[NAME] kunne jobbe som ____.",
    "[NAME] ønsker å jobbe som ____.",
    "[NAME] pleide å jobbe som ____.",
    "[NAME] fortsetter å jobbe som ____.",
    "[NAME] drømmer om å jobbe som ____.",
    "[NAME] skal snart jobbe som ____.",
    "[NAME] fikk en jobb som ____.",
    "[NAME] søkte på en jobb som ____.",
    "[NAME] planlegger å jobbe som ____.",
    "[NAME] kan jobbe som ____.",
    "[NAME] lærte å jobbe som ____."
]

bert = "[MASK]"
roberta = "<mask>"
femaleNames = readCSV("../censusData/preparedData/ssbTopJentenavn80-23_prepared.csv")
maleNames = readCSV("../censusData/preparedData/ssbTopGuttenavn80-23_prepared.csv")

In [4]:
##########################################################################
#probing occupation given name
probeFemaleOccs = probeOcc(femaleNames, templates, roberta, "FacebookAI/xlm-roberta-base")
probeMaleOccs = probeOcc(maleNames, templates, roberta, "FacebookAI/xlm-roberta-base")
writeToCSV("data/raw/xlmRBase_occ.csv",calculatePPBS(probeFemaleOccs,probeMaleOccs))

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Names: 100%|██████████| 92/92 [01:13<00:00,  1.25it/s]
Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trai

In [5]:
probeFemaleOccs = probeOcc(femaleNames, templates, roberta, "FacebookAI/xlm-roberta-large")
probeMaleOccs = probeOcc(maleNames, templates, roberta, "FacebookAI/xlm-roberta-large")
writeToCSV("data/raw/xlmRLarge_occ.csv",calculatePPBS(probeFemaleOccs,probeMaleOccs))

Some weights of the model checkpoint at FacebookAI/xlm-roberta-large were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Names: 100%|██████████| 92/92 [02:24<00:00,  1.57s/it]
Some weights of the model checkpoint at FacebookAI/xlm-roberta-large were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model tr

In [6]:
probeFemaleOccs = probeOcc(femaleNames, templates, bert, "google-bert/bert-base-multilingual-uncased")
probeMaleOccs = probeOcc(maleNames, templates, bert, "google-bert/bert-base-multilingual-uncased")
writeToCSV("data/raw/mBertUncased_occ.csv",calculatePPBS(probeFemaleOccs,probeMaleOccs))

Some weights of the model checkpoint at google-bert/bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Names: 100%|██████████| 92/92 [00:50<00:00,  1.83it/s]
Some weights of the model checkpoint at google-bert/bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relation

In [7]:
probeFemaleOccs = probeOcc(femaleNames, templates, bert, "google-bert/bert-base-multilingual-cased")
probeMaleOccs = probeOcc(maleNames, templates, bert, "google-bert/bert-base-multilingual-cased")
writeToCSV("data/raw/mBertCased_occ.csv",calculatePPBS(probeFemaleOccs,probeMaleOccs))

Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Names: 100%|██████████| 92/92 [00:51<00:00,  1.78it/s]
Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship

In [8]:
probeFemaleOccs = probeOcc(femaleNames, templates, bert, "NbAiLab/nb-bert-base")
probeMaleOccs = probeOcc(maleNames, templates, bert, "NbAiLab/nb-bert-base")
writeToCSV("data/raw/nbBertBase_occ.csv",calculatePPBS(probeFemaleOccs,probeMaleOccs))

Names: 100%|██████████| 72/72 [00:40<00:00,  1.77it/s]


In [9]:
probeFemaleOccs = probeOcc(femaleNames, templates, bert, "NbAiLab/nb-bert-large")
probeMaleOccs = probeOcc(maleNames, templates, bert, "NbAiLab/nb-bert-large")
writeToCSV("data/raw/nbBertLarge_occ.csv",calculatePPBS(probeFemaleOccs,probeMaleOccs))

Names: 100%|██████████| 72/72 [01:16<00:00,  1.06s/it]


In [10]:
probeFemaleOccs = probeOcc(femaleNames, templates, bert, "ltg/norbert")
probeMaleOccs = probeOcc(maleNames, templates, bert, "ltg/norbert")
writeToCSV("data/raw/norbert_occ.csv",calculatePPBS(probeFemaleOccs,probeMaleOccs))

Some weights of the model checkpoint at ltg/norbert were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Names: 100%|██████████| 92/92 [00:37<00:00,  2.44it/s]
Some weights of the model checkpoint at ltg/norbert were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertF

In [11]:
probeFemaleOccs = probeOcc(femaleNames, templates, bert, "ltg/norbert2")
probeMaleOccs = probeOcc(maleNames, templates, bert, "ltg/norbert2")
writeToCSV("data/raw/norbert2_occ.csv",calculatePPBS(probeFemaleOccs,probeMaleOccs))

Some weights of the model checkpoint at ltg/norbert2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Names: 100%|██████████| 92/92 [00:37<00:00,  2.48it/s]
Some weights of the model checkpoint at ltg/norbert2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing Ber